In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [2]:
df_train = pd.read_csv('F:/Titanic/data/train.csv')

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
def replace_age_fraction(df_train):
    df_train.loc[df_train['Age']<1,'Age'] = df_train.loc[df_train['Age']<1,'Age']*100
    print len(df_train.loc[df_train['Age']<1,'Age'])


In [5]:
def replace_na_to_mode(df_train):
    for i in df_train.columns:
    
        if(i in ['PassengerId','Name']):
            pass
        else:
            print i
            df_train[i]= df_train[i].fillna(df_train[i].mode()[0]) # fill na by mode Values

In [6]:
replace_age_fraction(df_train)


0


In [7]:
replace_na_to_mode(df_train)

Survived
Pclass
Sex
Age
SibSp
Parch
Ticket
Fare
Cabin
Embarked


### Create polynomial of column values

In [8]:
def polynomial_data(dataframe, column_name, power =1):
    tmp_df = pd.DataFrame()
    col_degree = [column_name+str(i) for i in range(1,power+1)]
    
    for i in range(1,power+1):
        tmp_df[col_degree[i-1]]=dataframe[column_name]**i
        
    return tmp_df

In [18]:

df_poly = pd.concat([polynomial_data(df_train,'Pclass',2), 
                 polynomial_data(df_train,'Age',2),
                 polynomial_data(df_train,'Fare',2), 
                 df_train], axis=1).head()

In [34]:
print len(df_train['Age'])
print len(df_poly['Age1'])

891
5


In [19]:
df_poly.columns

Index([u'Pclass1', u'Pclass2', u'Age1', u'Age2', u'Fare1', u'Fare2',
       u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

In [20]:
import h2o

In [21]:
h2o.init(max_mem_size='6g')

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 min 47 secs
H2O cluster version:,3.10.3.4
H2O cluster version age:,1 month and 1 day
H2O cluster name:,H2O_from_python_hamza_r1tklm
H2O cluster total nodes:,1
H2O cluster free memory:,5.312 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [31]:
hdf = h2o.H2OFrame(df_poly)
hdf['Survived']= hdf['Survived'].asfactor()
hdf_train, hdf_validate = hdf.split_frame([0.50], seed=100)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [32]:
hdf_validate.head(1)

Pclass1,Pclass2,Age1,Age2,Fare1,Fare2,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
3,9,22,484,7.25,52.5625,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,B96 B98,S


In [25]:
input_col = hdf.col_names
input_col.remove('Survived')
print input_col

[u'Pclass1', u'Pclass2', u'Age1', u'Age2', u'Fare1', u'Fare2', u'PassengerId', u'Pclass', u'Name', u'Sex', u'Age', u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked']


In [26]:
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator

In [27]:
nb =H2ONaiveBayesEstimator(seed =1)
nb.train(x =input_col, y = 'Survived', training_frame=hdf_train, validation_frame=hdf_validate)

H2OResponseError: ModelBuilderErrorV3  (water.exceptions.H2OModelBuilderIllegalArgumentException):
    stacktrace =
        water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for NaiveBayes model: NaiveBayes_model_python_1488664156061_1.  Details: ERRR on field: _validation_frame: Validation frame must have > 0 rows.
        water.exceptions.H2OModelBuilderIllegalArgumentException.makeFromBuilder(H2OModelBuilderIllegalArgumentException.java:20)
        hex.ModelBuilder.trainModel(ModelBuilder.java:194)
        water.api.ModelBuilderHandler.handle(ModelBuilderHandler.java:52)
        water.api.ModelBuilderHandler.handle(ModelBuilderHandler.java:16)
        water.api.RequestServer.serve(RequestServer.java:429)
        water.api.RequestServer.doGeneric(RequestServer.java:280)
        water.api.RequestServer.doPost(RequestServer.java:217)
        javax.servlet.http.HttpServlet.service(HttpServlet.java:755)
        javax.servlet.http.HttpServlet.service(HttpServle

In [ ]:
def model_performance(model =  nb):
    print 'Train accuracy:', model.auc(train=True)
    print 'valid accuracy:', model.auc(valid=True)

    roc_valid = model.roc(valid=True)
    roc_train= model.roc()

    plt.plot(roc_valid[0],roc_valid[1])
    plt.plot(roc_train[0],roc_train[1])
    plt.plot(np.arange(0.0,1.1,0.1),np.arange(0.0,1.1,0.1))
    
    plt.title('ROC Curve')
    plt.legend(['Valid','Train'])
    plt.show()

model_performance()

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [ ]:
gbm_model = H2OGradientBoostingEstimator(seed =1,  ntrees = 100, learn_rate = .1)
gbm_model.train(x =input_col, y = 'Survived', training_frame=hdf_train, validation_frame=hdf_validate)

In [ ]:
model_performance(gbm_model)

In [ ]:
gbm_model.plot()

In [ ]:
gbm_model1 = H2OGradientBoostingEstimator(seed =1,  ntrees = 15, learn_rate = .2003, distribution = 'AUTO', 
                                          build_tree_one_node = True)
gbm_model1.train(x =input_col, y = 'Survived', training_frame=hdf_train, validation_frame=hdf_validate)

In [ ]:
model_performance(gbm_model1)

In [ ]:
gbm_model1.plot()

## Predict with test data

In [ ]:
df_test = pd.read_csv('F:/Titanic/data/test.csv')

replace_age_fraction(df_test)
replace_na_to_mode(df_test)

In [ ]:
df_poly_test = pd.concat([polynomial_data(df_test,'Pclass',2), 
                 polynomial_data(df_test,'Age',5),
                 polynomial_data(df_test,'Fare',4), 
                 df_test], axis=1).head()

In [ ]:
# df_poly_test.head()

In [ ]:
hdf_test = h2o.H2OFrame(df_poly_test)

In [ ]:
def get_prediction(model= gbm_model, hdf_data = hdf_test, create_frame=True):
    prediction = model.predict(hdf_data)
    
    if(create_frame == True):
        lst_id=range(892,len(df_test['Age'])+892)

        #initialize dataframe and save prediction values in Survived columns and lst_id in Passenger id coloums
        df_submission = pd.DataFrame()
        df_submission['PassengerId'] = lst_id
        df_submission['Survived'] = prediction['predict'].as_data_frame()
        
        return df_submission

    else:
    #     prediction['predict'].unique()
        return prediction

In [ ]:
df_submission = get_prediction(gbm_model1,data=df_poly_test, create_frame=True)

In [ ]:
df_submission.to_csv('F:/Titanic/data/submission.csv',index=False)